In [15]:
import json
import random

class Character:
    def __init__(self):
        self.faction = ""
        self.styles = []
        self.values = []
        self.cliffhanger = []
        self.gender = ""
        
        self.chosen_styles = []
        self.assigned_values = []
        self.character_styles = []
        self.character_cliffhangers = []  # List for selected cliffhangers
        #self.select_styles = []
        self.faction_data = self.load_faction_data()
        
    def load_faction_data(self):
        with open("faction_data.json", "r", encoding="utf-8") as file:
            return json.load(file)

    def choose_faction(self):
        factions = list(self.faction_data.keys()) + ["Randomize Faction"]

        print("Choose your faction:")
        for i, faction in enumerate(factions, 1):
            print(f"{i}. {faction}")
        
        while True:
            try:
                choice = int(input("Enter the number of your choice: "))
                if 1 <= choice <= len(factions):
                    if factions[choice - 1] == "Randomize Faction":
                        self.random_faction_selection(factions[:-1])
                    else:
                        self.faction = factions[choice - 1]
                        print(f"You chose the {self.faction}!\n")
                    break
                else:
                    print("Please choose a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def random_faction_selection(self, factions):
        self.faction = random.choice(factions)
        print(f"Randomly selected faction: {self.faction}!\n")

    def faction_specific_styles(self):
        # Special gender handling for Borstenmütter
        if self.faction == "Borstenmütter":
            self.choose_gender()

        # Load styles and values for the chosen faction
        if self.faction in self.faction_data:
            # Handle Borstenmütter (gender-specific styles)
            if self.faction == "Borstenmütter" and self.gender:
                base_styles = self.faction_data[self.faction]["base_styles"][self.gender]["styles"]
                base_values = self.faction_data[self.faction]["base_styles"][self.gender]["values"]

            # For all other factions
            else:
                base_styles = self.faction_data[self.faction]["base_styles"]["styles"]
                base_values = self.faction_data[self.faction]["base_styles"]["values"]

            # Load additional categories like habit_styles and joker_styles
            habit_styles = self.faction_data[self.faction]["habit_styles"]["styles"]
            habit_values = self.faction_data[self.faction]["habit_styles"]["values"]

            joker_styles = self.faction_data[self.faction]["joker_styles"]["styles"]
            joker_values = self.faction_data[self.faction]["joker_styles"]["values"]

            # Merge all styles and values into one list
            self.styles = base_styles + habit_styles + joker_styles
            self.values = base_values + habit_values + joker_values
            

        
        else:
            print(f"No specific styles defined for {self.faction}.")

    def choose_gender(self):
        print("Borstenmütter: Bist du ein Keiler oder Bache?")
        while True:
            gender_input = input("Gib 'Keiler' oder 'Bache' ein: ").lower()
            if gender_input in ["Keiler", "Bache"]:
                self.gender = gender_input
                print(f"You chose {self.gender} Borstenmütter!\n")
                break
            else:
                print("Invalid input. Please enter 'Keiler' or 'Bache'.")
        

    def select_styles(self, style_category, count):
        """Select a given number of styles and assign values to them, displaying available options only once."""
        available_styles = self.faction_data[self.faction][style_category]["styles"]
        available_values = self.faction_data[self.faction][style_category]["values"]

        selected_styles = []
        selected_values = []

        # Display available styles only once
        print(f"\nWähle {len(available_values)} {style_category} mit den Werten {available_values}:")
        for i, style in enumerate(available_styles):
            print(f"{i + 1}. {style}")

        while len(selected_styles) < count:
            try:
                choice = int(input(f"\nChoose a {style_category} (1-{len(available_styles)}): ")) - 1
                if 0 <= choice < len(available_styles):
                    selected_style = available_styles[choice]
                    if selected_style not in selected_styles:
                        selected_styles.append(selected_style)
                        selected_values.append(self.assigned_values.pop(0))  # Assign the next available value
                        print(f"Selected: {selected_style}")
                    else:
                        print("You already selected this style.")
                else:
                    print("Invalid choice. Please choose a valid option.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Add selected styles and their assigned values as (style, value) tuples to character_styles
        self.character_styles.extend(zip(selected_styles, selected_values))

    def select_cliffhangers(self, count):
        """Select a given number of cliffhangers without assigning values, displaying available options only once."""
        available_cliffhangers = self.faction_data[self.faction]["cliffhanger"]
        selected_cliffhangers = []

        # Display available cliffhangers only once
        print("\n Wähle 2 Cliffhanger:")
        for i, cliffhanger in enumerate(available_cliffhangers):
            print(f"{i + 1}. {cliffhanger}")

        while len(selected_cliffhangers) < count:
            try:
                choice = int(input(f"\nChoose a cliffhanger (1-{len(available_cliffhangers)}): ")) - 1
                if 0 <= choice < len(available_cliffhangers):
                    selected_cliffhanger = available_cliffhangers[choice]
                    if selected_cliffhanger not in selected_cliffhangers:
                        selected_cliffhangers.append(selected_cliffhanger)
                        print(f"Selected: {selected_cliffhanger}")
                    else:
                        print("You already selected this cliffhanger.")
                else:
                    print("Invalid choice. Please choose a valid option.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        # Add selected cliffhangers to character_cliffhangers
        self.character_cliffhangers.extend(selected_cliffhangers)

    def choose_all_styles(self):
        """Handle three different style choices with corresponding values."""
        # Copy the values to be assigned for selection
        self.assigned_values = self.values[:]

        # Base styles selection
        base_style_count = len(self.faction_data[self.faction]["base_styles"]["values"])
        self.select_styles("base_styles", base_style_count)

        # Habit styles selection
        habit_style_count = len(self.faction_data[self.faction]["habit_styles"]["values"])
        self.select_styles("habit_styles", habit_style_count)

        # Joker styles selection
        joker_style_count = len(self.faction_data[self.faction]["joker_styles"]["values"])
        self.select_styles("joker_styles", joker_style_count)

        # Cliffhanger selection (without values)
        cliffhanger_count = 2  # You can change this number based on how many cliffhangers you want the user to choose
        self.select_cliffhangers(cliffhanger_count)

        self.display_style_summary()

    def display_style_summary(self):
        print("\nAssigned Styles (Ordered by Value):")
        # Sort character_styles by the second element in each tuple (value), in descending order
        sorted_styles = sorted(self.character_styles, key=lambda x: x[1], reverse=True)
    
        for style, value in sorted_styles:
            print(f"{style}: {value}")
        
        print("\nSelected Cliffhangers:")
        for cliffhanger in self.character_cliffhangers:
            print(f"- {cliffhanger}")


In [16]:
# Main execution
def main():
    character = Character()
    character.choose_faction()
    character.faction_specific_styles()
    character.choose_all_styles()

if __name__ == "__main__":
    main()

Choose your faction:
1. Architekten
2. Borstenmütter
3. Chronomaster
4. Cybersamurai
5. Corporat
6. Herren der Verlorenen See
7. Krieger von Valhalla
8. Kinder des Löwen
9. Mardukorden
10. Moon Circus
11. Nomads
12. Pan Doa
13. Schwesternschaft der 7
14. Der Schwarm
15. Überlebender
16. Union
17. Wanderer
18. Randomize Faction


Enter the number of your choice:  1


You chose the Architekten!


Wähle 2 base_styles mit den Werten [6, 6]:
1. Unbeholfener Nahkampf
2. Überlegene Ausstrahlung
3. Athletik
4. Geschulte Wahrnehmung
5. Perfekte Balance
6. Technologien hacken
7. Überzeugendes Charisma
8. Erhabener Reiter



Choose a base_styles (1-8):  1


Selected: Unbeholfener Nahkampf


...

Selected: Vollkommene Eleganz*

 Wähle 2 Cliffhanger:
1. Ich formiere den Nanoschwarm neu
2. Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm
3. Ich sage meine Grundsätze auf
4. Ich ziehe mich alleine zurück
5. Ich erzähle einen Witz



Choose a cliffhanger (1-5):  1


Selected: Ich formiere den Nanoschwarm neu



Choose a cliffhanger (1-5):  2


Selected: Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm

Assigned Styles (Ordered by Value):
Feiges Vorgehen-: 12
Unbeholfener Nahkampf: 6
Überlegene Ausstrahlung: 6
Vollkommene Eleganz*: 6

Selected Cliffhangers:
- Ich formiere den Nanoschwarm neu
- Ich reinige die Erde oder eine Wasserstelle mit meinem Nanoschwarm
